# Sklearn Workshop 2: Mobile Price Prediction (Classificaiton)


### Step 1: ETL

### Step 2: EDA

### Step 3: Splitting

### Step 4: Preprocessing

### Step 5: Modeling

### Step 6: Testing

### Step 7: Feature Importance

### Step 8: Saving the models